In [13]:
# import the necessary libraries here
from Crypto.Util import number
from Crypto.PublicKey import RSA as CryptoRSA
from Crypto.Util.number import long_to_bytes, bytes_to_long

In [14]:
class RSA:
    """Implements the RSA public key encryption / decryption."""

    def __init__(self, key_length):
        # define self.p, self.q, self.e, self.n, self.d here based on key_length
        # self.key = RSA.generate(key_length)
        self.key = CryptoRSA.generate(key_length)
        self.p = self.key.p
        self.q = self.key.q
        self.n = self.key.n
        self.e = self.key.e
        self.d = self.key.d

    def encrypt(self, message):
        # return encryption of binary_data here
        message_int = bytes_to_long(message)
        power = pow(message_int, self.e, self.n)
        return power

    def decrypt(self, encrypted_int_data):
        # return decryption of encrypted_binary_data here
        power = pow(encrypted_int_data, self.d, self.n)
        return long_to_bytes(power)

In [15]:
class RSAParityOracle(RSA):
    """Extends the RSA class by adding a method to verify the parity of data."""

    def is_parity_odd(self, encrypted_int_data):
        # Decrypt the input data and return whether the resulting number is odd
        decrypted_int = pow(encrypted_int_data, self.d, self.n)
        return decrypted_int % 2 == 1


Oracle in a way gives least significant bit each time when it says odd or even but it gives the leftmost bit of original text each time
C = P^e mod n

multiplying with 2^e on both sides gives
C1 = C*2^e = (2P)^e mod n
i.e., C1 is cipher of 2P
decrypting msg to get 2P

if rightmost bit is on i.e., oracle returns odd then P > N/2
otherwise bit is off i.e., oracle returns even then P < N/2

again checking for 4P now..
if even then P < N/4 or N/2 <= P < 3N/4
or if odd then N/4 <= P < N/2 or 3N/4 <= P < N

This way each turn doing like binary search we get P finally

In [16]:

def parity_oracle_attack(ciphertext, rsa_parity_oracle):
    # implement the attack and return the obtained plaintext
    e = rsa_parity_oracle.e
    n = rsa_parity_oracle.n
    left = 0
    right = n-1
    multiplier = pow(2, e, n)
    ciphertext2 = ciphertext
    while right - left >= 1:
        ciphertext = (ciphertext * multiplier) % n
        odd = rsa_parity_oracle.is_parity_odd(ciphertext)

        mid = (left + right) // 2
        if odd:
            left = mid
        else:
            right = mid
            
    for i in range(-256,256):
        if(rsa_parity_oracle.encrypt(long_to_bytes(left+i))==ciphertext2):
            left = left + i
            break
        
    return long_to_bytes(left)

        

In [17]:
def main():
    input_bytes = input("Enter the message: ")
    # input_bytes = "Hello, World!"

    # Generate a 1024-bit RSA pair
    rsa_parity_oracle = RSAParityOracle(1024)

    # Encrypt the message
    ciphertext = rsa_parity_oracle.encrypt(input_bytes.encode())
    print("Encrypted message is: ", ciphertext)
    
    decrypted = rsa_parity_oracle.decrypt(ciphertext)
    print("Directly decrypted text is: ",decrypted)
    
    
    # Check if the attack works
    plaintext = parity_oracle_attack(ciphertext, rsa_parity_oracle)
    print("Obtained plaintext: ", plaintext)
    # assert plaintext == input_bytes.encode()


if __name__ == '__main__':
    main()

Encrypted message is:  32159189184871209589809300976955032292717192533597116349144769699631906780238445173231703435203486045778309999617163350589934509417025532468748755963003663897250616295447079943975453368857126793303469869637364112280386853355980177407092764168365557545686469287472928583463380365214247300026216629719610240289
Directly decrypted text is:  b'hello WOrld~'
Obtained plaintext:  b'hello WOrld~'
